In [2]:
pip install "dash[testing]"

You should consider upgrading via the '/Users/trikushmehta/practice/quantium-starter-repo/venv/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [15]:
from dash import Dash, html, dcc
import plotly.express as px
import pandas as pd

In [16]:
df1 = pd.read_csv('data/daily_sales_data_0.csv')
df2 = pd.read_csv('data/daily_sales_data_1.csv')
df3 = pd.read_csv('data/daily_sales_data_2.csv')

# Concatenate the dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)
df = df[df['product'] == 'pink morsel']

In [17]:
# Calculate the total sales
df['price'] = df['price'].str.replace('$', '').astype(float)
df['sales'] = df['price'] * df['quantity']

In [18]:
df = df.drop(columns=['price', 'quantity', 'product'])

In [23]:
df['date']

0        2018-02-06
1        2018-02-06
2        2018-02-06
3        2018-02-06
28       2018-02-07
            ...    
41107    2022-02-13
41132    2022-02-14
41133    2022-02-14
41134    2022-02-14
41135    2022-02-14
Name: date, Length: 5880, dtype: object

In [20]:
df.to_csv('data/pink_morsel_sales.csv', index=False)

In [32]:
# Define the center date
center_date = pd.to_datetime('2021-01-15')
df['date'] = pd.to_datetime(df['date'])

# Filter ±7 days
mask = (df['date'] >= center_date - pd.Timedelta(days=7)) & (df['date'] <= center_date + pd.Timedelta(days=7))
df_filtered = df[mask]

In [35]:
app = Dash()
fig = px.line(
    df_filtered,
    x='date',
    y='sales',
    color='region',  # This creates one line per region!
    title='Sales Over Time by Region'
)

In [36]:
app.layout = html.Div([
    html.H1('Sales Dashboard'),
    dcc.Graph(
        id='sales-line-graph',
        figure=fig
    )
])
if __name__ == '__main__':
    app.run(debug=True)